# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [1]:
# Préambule
import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Acquisition et caractéristiques de la lumière

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

*Optics: Sert à concentrer la lumière, l'agrandir et la recentrer sur le capteur.*

*Aperture: Sert à agrandir ou rétrécir l'ouverture de la caméra pour gérer la quantité de lumière qui entre. (Quand il fait noir, ou flouter l'image)*

*Shutter: Sert à modifier le temps d'exposition du capteur à la lumière. (Si il fait noir, plus de temps d'exposition, si on bouge, plus le temps est long, plus l'objet va être flou)*

*Sensor: Sert à transformer les photons en signal électrique analogique.*

*Gain(ISO):Amplifie le signal analogique pour couvrir la totalité de la plage du dynamique du capteur. (Ça ajoute du bruit)*

*ADC:Permet de convertir le signal analogique en signal digital.*

*Demosaic: (Color filter array) Filtre de couleur pour améliorer la qualité de l'image. On passe par une matrice qui isole une couleur dans un canal et fait une moyenne des voisins pour reconstruire les couleurs manquante dans le canal. (Figure 2.31(a))*

*Denoise and sharpen: Permet de corriger le floutage du CFA et applique un filtre passe haut ou bas pour diminuer le bruit.*

*White balance: Ramener ce qui devait être blanc en un vrai blanc.*

*Gamma/curve: Permet de faire correspondre les valeurs numériques dans la facon que la valeur numérique est perçu par l'oeil. (Corrige la réponse naturelle de l'oeil)*

*Compress: Compresse l'image pour prendre moins de place.(Désavantage c'est qu'on dégrade l'information de l'image)*

*RAW: Permet de garder toute l'information de l'image et ça skip le post-processing parce qu'on peut faire mieux pour le post-processing*

*JPEG: Perte de la résolution de la couleur.*

### Q1.2

Quelle est la différence entre les paramètres intrinsèques et extrinsèques d'une caméra ? Décrivez chaque type en une phrase.

Intrinsèques: Ce qui est dans l'appareil. (Taille du capteur, la lentille, ...)
Extrinsèques: Ce qui est à l'extérieur de la caméra. (où elle est installé.)

### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ (Certains fabricants vont nous donner la matrice K, mais sinon, il faut faire la calibration pour la trouver) :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ? (Voir notes de cour page 58)

*La matrice K est uniquement en pixels, il faut donc les informations du capteur pour convertir en mm.*

*La raison pourquoi W et H sont divisé par deux c'est parce que c'est le centre de l'image.*

In [4]:
# Réponse ici.
K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]
])
Wp = K[0,2]*2
Hp = K[1,2]*2
Wm = 30/Wp  #(mm/px)
Hm = 15/Hp  #(mm/px)
fx = K[0,0]* Wm
fy = K[1,1]*Hm
print("Distance focale x: ", fx)
print("Distance focale y: ", fy)
print("Les pixels peuvent ne pas être carré ce qui donne deux distance focale en pixels différentes")


Distance focale x:  9.08203125
Distance focale y:  9.08203125
Les pixels peuvent ne pas être carré ce qui donne deux distance focale en pixels différentes


### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement. (p.79)

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

*Intrinsèque: Toutes les réalités d'une chaine de manufactures, défauts et autres comme dans la lentille, le boitier, ...*
*Extrinsèque: Une mauvaise mesure dans la position de la caméra dans l'environnement*

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

*Calibrer la caméra avec un outil de calibration comme le panneaux à damiers, ...*

### Q1.5

Dans cette image synthétique (p.67):

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

*Diffuse: Les parties jaune de l'image, c'est ce qui donne la couleur.*
*Spéculaire: Les parties qui sont très blanches. (Gloss or Higlight)*

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Vous pouvez répondre en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

*Ce n'est pas possible, car ça implique beaucoup de données qu'on ne connait pas*
*On peut avec les deux boules estimer la position de la lumière en joignant les deux vecteurs normaux des points blancs de la réflexion spéculaire.*
*Avec une caméra stéréoscopique*

### Q1.6

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

*S'ils ne sont pas calibrer de la bonne façon, les capteurs ont des tolérances et ainsi de suite, donc il est peu probable*

**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

*Avec l'espace de couleur x,y,z. (Voir figure 2.30)*
*On peut donc comparer si on a les deux matrices de conversion vers le x,y,z des deux capteurs*
*Le lab: ressemble à x,y,z, mais il as été conçu pour que la distance euclidienne soit proportionnelle à ce que l'oeil perçoit.*

## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [9]:
# Aligner le repère sur l'autre pour avoir la matrice de rotation et ensuite la dernière colonne est la translation.
T_10 = np.array([[0, 1, 0, .24],
                 [1, 0, 0, 0.8],
                 [0, 0, -1, 0.12],
                 [0, 0, 0, 1]])
print("T_10:\n", T_10)

T_10:
 [[ 0.    1.    0.    0.24]
 [ 1.    0.    0.    0.8 ]
 [ 0.    0.   -1.    0.12]
 [ 0.    0.    0.    1.  ]]


**b)** Trouvez maintenant la transformation inverse.

In [11]:
T_01 = np.linalg.inv(T_10)
print("T_01:\n", T_01)

T_01:
 [[ 0.    1.    0.   -0.8 ]
 [ 1.    0.    0.   -0.24]
 [-0.   -0.   -1.    0.12]
 [ 0.    0.    0.    1.  ]]


**c)** Soit le point $p_0 = [8, 5, 1]^T$, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [14]:
# On ajoute un 1 à la fin pour permettre de faire la translation et l'inversion de la matrice.
p_0 = [8, 5, 1, 1]
p_1 = T_01 @ p_0
print("P_1:\n", p_1)

P_1:
 [ 4.2   7.76 -0.88  1.  ]


### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$. (équation 2.64)

In [40]:
z = T_10[2,3]
R = T_01[0:3,0:3]
t = T_01[0:3,3]
Zero = np.zeros((3,1))
K_ = np.array([[K[0,0], K[0,1], K[0,2], 0],
               [K[1,0], K[1,1], K[1,2], 0],
               [K[2,0], K[2,1], K[2,2], 0],
               [0, 0, 0, 1]])
print("K_:\n", K_)

P = K_ @ T_01
print("P:\n", P)

K_:
 [[6.200e+02 0.000e+00 1.024e+03 0.000e+00]
 [0.000e+00 6.200e+02 5.120e+02 0.000e+00]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00]]
P:
 [[ 0.0000e+00  6.2000e+02 -1.0240e+03 -3.7312e+02]
 [ 6.2000e+02  0.0000e+00 -5.1200e+02 -8.7360e+01]
 [ 0.0000e+00  0.0000e+00 -1.0000e+00  1.2000e-01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]


**b)** Soit le point $p_0 = [0.250, 0.010, 0.000]$. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra. (Équation 2.65)

In [41]:
p_0 = np.array([0.250, 0.010, 0.000, 1])
x_s = P @ p_0
print("x_s:\n", x_s)

x_s:
 [-3.6692e+02  6.7640e+01  1.2000e-01  1.0000e+00]


On y est presque, il suffit de

In [42]:
xs = x_s/z
print("xs:\n", xs)

xs:
 [-3.05766667e+03  5.63666667e+02  1.00000000e+00  8.33333333e+00]


## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [ ]:
l_conv = 0


### Q3.2

Soit le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [38]:
disparity = 1/z
x_s = np.array([120,200,1,disparity])
x_s = x_s * z
print("x_s:\n", x_s)

x_s:
 [14.4  24.    0.12  1.  ]
